In [10]:
""""This file almost exclusively written by chatgpt
as its only purpose is exploring tons of options"""
import itertools as it
import numpy as np
gems = [2, 3, 0, 4, 5]

In [22]:
import itertools
import tensorflow as tf

# Number of gems and how many gems we want to take
n_gems = 5
k      = 3

# Step 1: Enumerate all combinations of length k
combs = list(itertools.combinations(range(n_gems), k))
# e.g., [(0,1,2), (0,1,3), (0,1,4), (0,2,3), ..., (2,3,4)]

# Step 2: Convert the Python list to a Tensor
combs_tf = tf.constant(combs, dtype=tf.int32)  # shape = (n_combs, k)

# Step 3: Create one-hot vectors and sum along the combination dimension
# onehots will have shape = (n_combs, k, n_gems)
onehots = tf.one_hot(combs_tf, depth=n_gems, axis=-1, dtype=tf.int32)

# Summing along axis=1 collapses each k-combination into a length-n_gems row
takes = tf.reduce_sum(onehots, axis=1)  # shape = (n_combs, n_gems)

print(takes.numpy())


[[1 1 1 0 0]
 [1 1 0 1 0]
 [1 1 0 0 1]
 [1 0 1 1 0]
 [1 0 1 0 1]
 [1 0 0 1 1]
 [0 1 1 1 0]
 [0 1 1 0 1]
 [0 1 0 1 1]
 [0 0 1 1 1]]


In [ ]:
# Now form all pairs (pick_i, discard_j) by broadcasted subtraction
# takes[:, None, :] => shape (num_combos, 1, n_gems)
# takes[None, :, :] => shape (1, num_combos, n_gems)
# Subtract => shape (num_combos, num_combos, n_gems)
pick_discard_matrix = takes[:, None, :] - takes[None, :, :]

# If desired, flatten from (num_combos, num_combos, n_gems) to (num_combos^2, n_gems)
pick_discard_matrix = tf.reshape(pick_discard_matrix, (-1, n_gems))
pick_discard_matrix

In [57]:
n_gems = 5
k_pick = 2
k_discard = 2

# 1. Enumerate picks (3 distinct)
picks_list = list(itertools.combinations(range(n_gems), k_pick))
picks_tf = tf.constant(picks_list, dtype=tf.int32)  # (P, k_pick)
picks_mat = tf.reduce_sum(tf.one_hot(picks_tf, depth=n_gems, axis=-1, dtype=tf.int32), axis=1) 
# shape => (P, n_gems)

# 2. Enumerate discards (3 gems, possibly repeating)
discards_list = list(itertools.combinations_with_replacement(range(n_gems), k_discard))
discards_tf = tf.constant(discards_list, dtype=tf.int32)  # (D, k_discard)
discards_mat = tf.reduce_sum(tf.one_hot(discards_tf, depth=n_gems, axis=-1, dtype=tf.int32), axis=1)
# shape => (D, n_gems)

# 3. Combine via broadcasted subtraction => (P, D, n_gems)
combined_3D = picks_mat[:, None, :] - discards_mat[None, :, :]
combined_2D = tf.reshape(combined_3D, (-1, n_gems))  # shape = (P*D, n_gems)

print("Broadcasted shape:", combined_2D.shape)
print("Sample combined rows:\n", combined_2D[10:20].numpy())

# 4. Optionally filter out invalid or undesired rows
mask_valid = tf.reduce_all(combined_2D >= 0, axis=1)
valid_rows = tf.boolean_mask(combined_2D, mask_valid)

print("\nNumber of valid rows:", valid_rows.shape[0])
print("Sample valid row:\n", valid_rows[:1].numpy())


Broadcasted shape: (150, 5)
Sample combined rows:
 [[ 1  1 -1 -1  0]
 [ 1  1 -1  0 -1]
 [ 1  1  0 -2  0]
 [ 1  1  0 -1 -1]
 [ 1  1  0  0 -2]
 [-1  0  1  0  0]
 [ 0 -1  1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  1 -1  0]
 [ 0  0  1  0 -1]]

Number of valid rows: 10
Sample valid row:
 [[0 0 0 0 0]]


In [58]:
for i in combined_2D:
    print(i)

tf.Tensor([-1  1  0  0  0], shape=(5,), dtype=int32)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int32)
tf.Tensor([ 0  1 -1  0  0], shape=(5,), dtype=int32)
tf.Tensor([ 0  1  0 -1  0], shape=(5,), dtype=int32)
tf.Tensor([ 0  1  0  0 -1], shape=(5,), dtype=int32)
tf.Tensor([ 1 -1  0  0  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  0 -1  0  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  0  0 -1  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  0  0  0 -1], shape=(5,), dtype=int32)
tf.Tensor([ 1  1 -2  0  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  1 -1 -1  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  1 -1  0 -1], shape=(5,), dtype=int32)
tf.Tensor([ 1  1  0 -2  0], shape=(5,), dtype=int32)
tf.Tensor([ 1  1  0 -1 -1], shape=(5,), dtype=int32)
tf.Tensor([ 1  1  0  0 -2], shape=(5,), dtype=int32)
tf.Tensor([-1  0  1  0  0], shape=(5,), dtype=int32)
tf.Tensor([ 0 -1  1  0  0], shape=(5,), dtype=int32)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int32)
tf.Tensor([ 0  0  1 -1  0], shape=(5,), dtype=int32)
tf.